<a href="https://colab.research.google.com/github/skmisht/CNN/blob/main/CNN_Caffe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Melanoma detection using - Caffe

In [3]:
#sample caffe cnn sample config https://github.com/BVLC/caffe/blob/master/examples/net_surgery/bvlc_caffenet_full_conv.prototxt
#Connect google colab to google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!kill -9 -1

In [1]:
with open('/etc/apt/sources.list') as f:
  txt = f.read()
with open('/etc/apt/sources.list.backup', 'w') as f:
  f.write(txt)
with open('/etc/apt/sources.list', 'w') as f:
  f.write(txt.replace('# deb-src','deb-src'))

In [ ]:
!apt update
!apt build-dep caffe-cuda
!apt install g++-5

In [ ]:
!wget https://dl.bintray.com/boostorg/release/1.67.0/source/boost_1_67_0.tar.bz2
!tar --bzip2 -xf boost_1_67_0.tar.bz2

In [ ]:
!update-alternatives --remove-all gcc 
!update-alternatives --remove-all g++

!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 20
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-5 20

!update-alternatives --install /usr/bin/cc cc /usr/bin/gcc 30
!update-alternatives --set cc /usr/bin/gcc

!update-alternatives --install /usr/bin/c++ c++ /usr/bin/g++ 30
!update-alternatives --set c++ /usr/bin/g++

In [ ]:
!cd boost_1_67_0 && ./bootstrap.sh --exec-prefix=/usr/local --with-libraries=system,filesystem,regex,thread,python \
--with-python-version=2.7 --with-python-root=/usr
!cd boost_1_67_0 && ./b2 install

In [7]:
!git clone https://github.com/weiliu89/caffe.git && cd caffe && git checkout ssd

Cloning into 'caffe'...
remote: Enumerating objects: 30179, done.
remote: Total 30179 (delta 0), reused 0 (delta 0), pack-reused 30179
Receiving objects: 100% (30179/30179), 48.27 MiB | 18.09 MiB/s, done.
Resolving deltas: 100% (19451/19451), done.
Branch 'ssd' set up to track remote branch 'ssd' from 'origin'.
Switched to a new branch 'ssd'


In [8]:
with open('caffe/Makefile.config.example') as f:
  config = f.read()
  
comment = ['CUDA_DIR := /usr/local/cuda', 
           'BLAS := open']
uncomment = ['# CUDA_DIR := /usr', 
             '# BLAS := atlas', 
             '# OPENCV_VERSION := 3', '# WITH_PYTHON_LAYER := 1'] #
replace = [('INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include',
            'INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include /usr/include/hdf5/serial /usr/local/lib/python2.7/dist-packages/numpy/core/include/'), 
           ('LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib',
            'LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib /usr/lib/x86_64-linux-gnu/hdf5/serial')]

for c in uncomment:
  config = config.replace(c, c[2:])
for c in comment:
  config = config.replace(c, '# '+c)
for c1,c2 in replace:
  config = config.replace(c1, c2)
  
with open('caffe/Makefile.config', 'w') as f:
  f.write(config)

In [9]:
with open('caffe/Makefile') as f:
  mfile = f.read()
  
with open('caffe/Makefile.backup', 'w') as f:
  f.write(mfile)
  
with open('caffe/Makefile', 'w') as f:
  f.write(mfile.replace('-isystem','-I'))

In [11]:
with open('/usr/include/x86_64-linux-gnu/c++/6/bits/c++config.h') as f:
  txt = f.read()
with open('/usr/include/x86_64-linux-gnu/c++/6/bits/c++config.h', 'w') as f:
  f.write(txt.replace('/* #undef _GLIBCXX_USE_C99_MATH */',
                      '/* #undef _GLIBCXX_USE_C99_MATH */\n#define  _GLIBCXX_USE_C99_MATH  1'))

In [12]:
!cd caffe && make -j8 && make pycaffe && make test -j8 && make distribute

CXX src/caffe/data_reader.cpp
PROTOC src/caffe/proto/caffe.proto
CXX src/caffe/layer.cpp
CXX src/caffe/internal_thread.cpp
CXX src/caffe/common.cpp
CXX src/caffe/layer_factory.cpp
CXX src/caffe/data_transformer.cpp
CXX src/caffe/parallel.cpp
src/caffe/parallel.cpp:2:10: fatal error: cuda_runtime.h: No such file or directory
 #include <cuda_runtime.h>
          ^~~~~~~~~~~~~~~~
compilation terminated.
Makefile:575: recipe for target '.build_release/src/caffe/parallel.o' failed
make: *** [.build_release/src/caffe/parallel.o] Error 1
make: *** Waiting for unfinished jobs....
In file included from /usr/include/cublas_v2.h:65:0,
                 from ./include/caffe/util/device_alternate.hpp:34,
                 from ./include/caffe/common.hpp:19,
                 from ./include/caffe/blob.hpp:8,
                 from ./include/caffe/data_transformer.hpp:8,
                 from src/caffe/data_transformer.cpp:8:
/usr/include/cublas_api.h:72:10: fatal error: driver_types.h: No such file or d

In [14]:
!cd caffe/python && pwd

/content/caffe/python


In [5]:
#install caffe with GPU support
#!apt install -y caffe-cuda
#install caffe with CPU support
!apt install -y caffe-cpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
caffe-cpu is already the newest version (1.0.0-6).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [10]:
import pandas as pd 

training_truth_data = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Part3_GroundTruth.csv') 
testing_truth_data = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Part3_GroundTruth.csv') 

training_metadata = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC-2017_Training_Data_metadata.csv')
testing_metadata = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data_metadata.csv')

path_training_images ='/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/'
path_testing_images ='/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/'


#add new column to training_truth_data witht he path to corresponding image
path_to_training_images_array = []
for x in training_truth_data.values:
    path_to_image = path_training_images + x[0] +'.jpg'
    path_to_training_images_array.append(path_to_image)
#add path to image to the training data frame
training_truth_data['path_to_image'] = path_to_training_images_array
#merge training truth data with metadata, join on the image_id's
training_data = pd.merge(training_truth_data, training_metadata, left_on='image_id', right_on='image_id', how='left').drop('image_id', axis=1)

#add new column to test_truth_data witht he path to corresponding image
path_to_test_images_array = []
for x in testing_truth_data.values:
  path_to_image = path_testing_images + str(x[0])+'.jpg'
  path_to_test_images_array.append(path_to_image)
#add path to image to the testing data frame
testing_truth_data['path_to_image'] = path_to_test_images_array
#merge testing truth data with metadata, join on the image_id's
testing_data = pd.merge(testing_truth_data, testing_metadata, left_on='image_id', right_on='image_id', how='left').drop('image_id', axis=1)


print(training_data.head())
print(testing_data.head())







   melanoma  seborrheic_keratosis  ... age_approximate     sex
0       0.0                   0.0  ...              55  female
1       0.0                   0.0  ...              30  female
2       1.0                   0.0  ...              60  female
3       0.0                   0.0  ...              30    male
4       1.0                   0.0  ...              80    male

[5 rows x 5 columns]
   melanoma  seborrheic_keratosis  ... age_approximate     sex
0       0.0                   1.0  ...              30    male
1       0.0                   0.0  ...              40    male
2       0.0                   0.0  ...              25    male
3       0.0                   1.0  ...              75  female
4       0.0                   1.0  ...              70    male

[5 rows x 5 columns]


In [23]:
text = []
for x in training_data.values:
  text.append(x[2]+' '+str(x[0])+'\n')

with open ('/content/drive/MyDrive/Caffe/train.txt', 'w') as T:
  T.writelines(text)

In [20]:
import sys
import h5py, os
caffe_path = !cd caffe/python && pwd
sys.path.insert(0, caffe_path[0])  
import caffe.io

import numpy as np
SIZE = 224 # fixed size to all images
with open( '/content/drive/MyDrive/Caffe/train.txt', 'r' ) as T :
 lines = T.readlines()
# If you do not have enough memory split data into
# multiple batches and generate multiple separate h5 files
X = np.zeros( (len(lines), 3, SIZE, SIZE), dtype='f4' )
y = np.zeros( (1,len(lines)), dtype='f4' )
for i,l in enumerate(lines):
 sp = l.split(' ')
 img = caffe.io.load_image(sp[0])
 img = caffe.io.resize( img, (SIZE, SIZE, 3) ) # resize to fixed size
 # you may apply other input transformations here...
 # Note that the transformation should take img from size-by-size-by-3 and transpose it to3-by-size-by-size
 X[i] = img
 y[i] = float(sp[1])
with h5py.File('/content/drive/MyDrive/Caffe/train.h5','w') as H:
 H.create_dataset( 'X', data=X )

ModuleNotFoundError: ignored

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import caffe

#set mode to use GPU
caffe.set_mode_cpu()

#load network architecture
net = caffe.Net('/content/drive/MyDrive/Caffe/cnn_model.prototxt', caffe.TEST)

#2351622
input = None
for x in training_data.values:
  im = np.array(Image.open(x[2]))
  im_input = im[np.newaxis, np.newaxis, :, :]
  print(x[2] +'  '+str(im_input.size))
  if (input is None):
    input = im_input
  else:
    input = np.concatenate((input, im_input))

net.blobs['data'].reshape(*input.shape)
net.blobs['data'].data[...] = input

net.save('/content/drive/MyDrive/Caffe/melanoma_cnn.caffemodel')



AttributeError: ignored

ModuleNotFoundError: ignored